# Extract cyclone windspeed return period maps

In [ ]:
import os
import re
from glob import glob
from pathlib import Path

import rasterio
import rioxarray
import xarray

In [ ]:
os.chdir("../../incoming_data/processed_swaths/")
out_dir = "../../processed_data/hazards/chaz_cyclones"
Path(out_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
fnames = sorted(glob("**/*.nc", recursive=True))
fnames

In [ ]:
def read_netcdf(fname):
    ds = xarray.open_dataset(fname)
    del ds['file']
    ds = ds.rename({'lat':'y','lon':'x'})
    ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
    ds.rio.write_crs("epsg:4326", inplace=True)
    return ds

In [ ]:
RPS = [5, 10, 50, 100]
for fname in fnames:
    matches = re.search(r"(\w+)/max_windspeed_(\d+)", fname)
    ssp, year = matches.groups()

    if "future" in ssp:
        ssp = f'ssp{ssp.replace("_future", "")}'
    else:
        ssp = "baseline"

    if year == "1951":
        epoch = "2010"
    elif year == "2030":
        epoch = "2030"
    elif year == "2060":
        epoch = "2050"
    elif year == "2080":
        continue
    else:
        assert False, "Unrecognised year"

    
    ds = read_netcdf(fname)
    for rp in RPS:
        out_fname  = os.path.join(
            out_dir, 
            f"cyclone_windspeed__epoch_{epoch}__rcp_{ssp}__rp_{rp}__isoa3_lca.tif")
        ds[f'rp_{rp}'].rio.to_raster(out_fname)
        print(out_fname)